Я решил выделить обучение и сохранение моделей в отдельный ноутбук, как будто этоспециальный сервис для этого (например ML Flow)

Написать классы-обертки вокруг моделей sklern чтобы инкапсулировать предобработку данных, сохранение и загрузку данных и сделать модель синглтоном, чтобы не загружать модель несколько раз. + Решил сделать API отличное от sklearn, но более подходящее для детекции атипичного поведения

In [1]:
from pathlib import Path

import pandas as pd
from sklearn.metrics import recall_score

from models.mean_based_models import MeanKNNModel
from models.embeddings_based_model import EmbenddingsKNNModel, EmbenddingsLinearModel
from models.utils import train_test_split_by_hours

In [2]:
ANOMALY_START = "2020-01-25"

In [3]:
df = pd.read_csv("../data/database_gas.csv")
df["timestamp"] = pd.to_datetime(df["timestamp"]).round('1s').dt.tz_localize(None)
df

,timestamp,temperature,humidity,CO2CosIRValue,CO2MG811Value,MOX1,MOX2,MOX3,MOX4,COValue
0,2019-11-06 11:37:13,19.48,54.86,128.0,563.0,476.0,731.0,649.0,565.0,128.0
1,2019-11-06 11:37:33,19.59,54.23,129.0,563.0,477.0,731.0,649.0,565.0,125.0
2,2019-11-06 11:37:53,19.63,54.05,128.0,566.0,478.0,732.0,649.0,565.0,125.0
3,2019-11-06 11:38:13,19.64,53.74,128.0,566.0,478.0,732.0,649.0,565.0,125.0
4,2019-11-06 11:38:33,19.67,53.53,128.0,569.0,480.0,732.0,650.0,565.0,125.0
...,...,...,...,...,...,...,...,...,...,...
416148,2020-02-13 11:56:54,20.62,57.75,62.0,518.0,524.0,698.0,642.0,591.0,157.0
416149,2020-02-13 11:57:14,20.62,57.75,62.0,518.0,524.0,698.0,641.0,591.0,157.0
416150,2020-02-13 11:57:34,20.62,57.75,62.0,518.0,524.0,698.0,641.0,591.0,157.0
416151,2020-02-13 11:57:54,20.62,57.74,63.0,518.0,524.0,698.0,641.0,591.0,157.0


In [4]:
# Разделяем данные на типичные и нетипичные
typical_data = df.query("timestamp < @ANOMALY_START")
atypical_data = df.query("timestamp >= @ANOMALY_START")

In [5]:
# Для проверки, что все работает, и оценки метрик разобъем на трейн и тест типичные и атипичные данные
typical_data_train, typical_data_test = train_test_split_by_hours(typical_data, 
                                                                  test_size=0.3,
                                                                  random_state=1)

atypical_data_train, atypical_data_test = train_test_split_by_hours(atypical_data, 
                                                                    test_size=0.3, 
                                                                    random_state=1)

data_test = pd.concat([typical_data_test, atypical_data_test]).dropna()
data_test

,timestamp,temperature,humidity,CO2CosIRValue,CO2MG811Value,MOX1,MOX2,MOX3,MOX4,COValue
0,2019-11-06 11:37:13,19.48,54.86,128.0,563.0,476.0,731.0,649.0,565.0,128.0
1,2019-11-06 11:37:33,19.59,54.23,129.0,563.0,477.0,731.0,649.0,565.0,125.0
2,2019-11-06 11:37:53,19.63,54.05,128.0,566.0,478.0,732.0,649.0,565.0,125.0
3,2019-11-06 11:38:13,19.64,53.74,128.0,566.0,478.0,732.0,649.0,565.0,125.0
4,2019-11-06 11:38:33,19.67,53.53,128.0,569.0,480.0,732.0,650.0,565.0,125.0
...,...,...,...,...,...,...,...,...,...,...
415793,2020-02-13 09:58:35,20.54,57.48,44.0,518.0,533.0,706.0,651.0,601.0,149.0
415794,2020-02-13 09:58:55,20.54,57.48,44.0,518.0,532.0,706.0,650.0,601.0,149.0
415795,2020-02-13 09:59:15,20.54,57.48,44.0,518.0,532.0,706.0,650.0,601.0,150.0
415796,2020-02-13 09:59:35,20.50,57.55,44.0,518.0,532.0,706.0,650.0,601.0,149.0


In [6]:
model = MeanKNNModel()
model.fit(typical_data_train, atypical_data_train)
y_pred = model.predict(data_test.dropna())
y_pred["anomaly_fact"] = (y_pred["timestamp"] >= ANOMALY_START).astype(int)
recall_score(y_pred["anomaly_fact"], y_pred["anomaly_prediction"])

0.9538461538461539

In [7]:
model = EmbenddingsKNNModel()
model.fit(typical_data_train, atypical_data_train)
y_pred = model.predict(data_test.dropna())
y_pred["anomaly_fact"] = (y_pred["timestamp"] >= ANOMALY_START).astype(int)
recall_score(y_pred["anomaly_fact"], y_pred["anomaly_prediction"])

0.96875

In [8]:
model = EmbenddingsLinearModel()
model.fit(typical_data_train, atypical_data_train)
y_pred = model.predict(data_test.dropna())
y_pred["anomaly_fact"] = (y_pred["timestamp"] >= ANOMALY_START).astype(int)
recall_score(y_pred["anomaly_fact"], y_pred["anomaly_prediction"])

0.9765625

Все супер, все работает как и ожидалось! Остается обучить модели на всех данных и сохранить их!

In [9]:
model = MeanKNNModel()
model.fit(typical_data, atypical_data)
model.save(Path("../models/mean_knn_model.pkl"))

In [10]:
model = EmbenddingsKNNModel()
model.fit(typical_data, atypical_data)
model.save(Path("../models/embenddings_knn_model.pkl"))

In [11]:
model = EmbenddingsLinearModel()
model.fit(typical_data, atypical_data)
model.save(Path("../models/embenddings_linear_model.pkl"))